# Interactive Trading Dashboard Prototype

This notebook demonstrates constructing a small interactive analytics dashboard (Panel + Plotly) and satisfies the requirement of having at least one executed cell.

Sections follow the prescribed outline.

In [ ]:
# 1. Set Up Environment and Imports
# (If panel not installed in this environment, uncomment the %pip line below.)
# %pip install panel bokeh plotly seaborn --quiet
import importlib, sys, math, json, os, random, statistics
import pandas as pd, numpy as np
import datetime as dt

try:
    import panel as pn
    import plotly.express as px
except ImportError as exc:
    raise RuntimeError("Panel or Plotly not available. Install with %pip install panel plotly") from exc

pn.extension('plotly')

PANEL_VERSION = importlib.metadata.version('panel') if hasattr(importlib, 'metadata') else getattr(pn, '__version__', 'unknown')
PLOTLY_VERSION = importlib.import_module('plotly').__version__
PANDAS_VERSION = pd.__version__
NUMPY_VERSION = np.__version__

print(f"Versions -> panel:{PANEL_VERSION} plotly:{PLOTLY_VERSION} pandas:{PANDAS_VERSION} numpy:{NUMPY_VERSION}")

In [ ]:
# 2. Load Sample Dataset
# Create synthetic multi-category time series (30 days * 4 categories)
np.random.seed(42)
base_date = dt.date.today() - dt.timedelta(days=29)
rows = []
categories = ["Momentum", "Value", "Growth", "Macro"]
for i in range(30):
    d = base_date + dt.timedelta(days=i)
    for cat in categories:
        val = 100 + i * np.random.uniform(0.2, 1.5) + np.random.normal(0, 2.5)
        rows.append((d, cat, float(val)))

df = pd.DataFrame(rows, columns=["date", "category", "value"])

In [ ]:
# 3. Data Preparation Utilities

def make_date_range(frame: pd.DataFrame):
    return frame['date'].min(), frame['date'].max()

def filter_frame(frame: pd.DataFrame, categories_sel, date_range):
    if not categories_sel:
        filtered = frame.copy()
    else:
        filtered = frame[frame['category'].isin(categories_sel)].copy()
    start, end = date_range
    mask = (filtered['date'] >= start) & (filtered['date'] <= end)
    return filtered.loc[mask]

def aggregate_frame(frame: pd.DataFrame, freq: str):
    if frame.empty:
        return frame
    # Convert to datetime for resampling
    tmp = frame.copy()
    tmp['date_dt'] = pd.to_datetime(tmp['date'])
    rule = {'Daily': 'D', 'Every2D': '2D', 'Weekly': 'W'}.get(freq, 'D')
    agg = tmp.set_index('date_dt').groupby('category')['value'].resample(rule).mean().reset_index()
    agg.rename(columns={'date_dt': 'date'}, inplace=True)
    agg['date'] = agg['date'].dt.date
    return agg

In [ ]:
# 4. Executed Sanity Check Cell
print("DataFrame shape:", df.shape)
print(df.head())
assert not df.empty, "DataFrame unexpectedly empty"
assert pd.api.types.is_numeric_dtype(df['value']), "Value column must be numeric"
print("Sanity checks passed ✅")

In [ ]:
# 5. Define Reactive Data Selectors (Panel Widgets)
min_date, max_date = make_date_range(df)
category_select = pn.widgets.MultiSelect(name='Categories', options=categories, value=categories[:2], size=5)
date_slider = pn.widgets.DateRangeSlider(name='Date Range', start=min_date, end=max_date, value=(min_date, max_date))
freq_radio = pn.widgets.RadioButtonGroup(name='Frequency', options=['Daily','Every2D','Weekly'], value='Daily')

In [ ]:
# 6. Create Plots Function

def make_plot(categories_sel, date_rng, freq):
    f = filter_frame(df, categories_sel, date_rng)
    agg = aggregate_frame(f, freq)
    if agg.empty:
        return px.line(title="No data for selection")
    fig = px.line(agg, x='date', y='value', color='category', title='Category Performance')
    fig.update_layout(legend_orientation='h', legend_y=-0.2, height=420, template='plotly_dark')
    return fig

In [ ]:
# 7. Compose Dashboard Layout
plot_bind = pn.bind(make_plot, categories_sel=category_select, date_rng=date_slider, freq=freq_radio)

@pn.depends(category_select, date_slider, freq_radio)
def stats_view(categories_sel, date_rng, freq):
    filt = filter_frame(df, categories_sel, date_rng)
    if filt.empty:
        return pn.pane.Markdown('**No data**')
    return pn.pane.Markdown(f"""
**Stats**  
Rows: {len(filt)}  
Mean: {filt['value'].mean():.2f}  
Std: {filt['value'].std():.2f}  
Min: {filt['value'].min():.2f}  
Max: {filt['value'].max():.2f}
""")

sidebar = pn.Column("### Controls", category_select, date_slider, freq_radio, sizing_mode='stretch_width')
main_col = pn.Column(pn.Row(pn.panel(plot_bind, sizing_mode='stretch_both')), stats_view, sizing_mode='stretch_both')

template = pn.template.MaterialTemplate(title="Trading Analytics Prototype", sidebar=[sidebar])
template.main.append(main_col)

template

In [ ]:
# 8. Add Callback Bindings and Interactivity
# (Already using pn.bind and @pn.depends above.)
# Display reactive objects explicitly (template already captures them).
_ = (plot_bind, stats_view)
print("Interactivity wired: plot and stats auto-update on widget changes.")

In [ ]:
# 9. Serve Dashboard (Instructions)
print("To serve this dashboard externally run:")
print("  panel serve dashboard_usage.ipynb --show --autoreload")
print("Or inside Python script: pn.serve(template, show=True)")
# Make the template servable in panel context
template.servable();

In [ ]:
# 10. Lightweight Function Test Cell
sample = filter_frame(df, categories[:1], (min_date, max_date))
assert not sample.empty, "Filtered sample unexpectedly empty"
assert sample['value'].dtype.kind in 'fc', "Numeric type expected for value column"
print("Function tests passed ✅  Rows=", len(sample))